# IT Service Ticket Classification

## Navegação dos notebooks

- `notebooks/analysis.ipynb`: análise exploratória (EDA).
- `notebooks/classificators.ipynb`: testes de classificadores (TF-IDF, embeddings, RAG) com métricas no conjunto de teste.
- `notebooks/main.ipynb`: visão geral, prompts, pipeline, avaliação final (usa o classificador escolhido).


## 1. Carregamento e Preparação dos Dados

### Divisão em treino, teste e validação

Usamos três conjuntos de dados:

1. **Validação (balanceada)**: `VALIDATION_SIZE` tickets, com o mesmo número de exemplos por classe.
2. **Treino/Teste**: o restante é dividido em **80% treino** e **20% teste**, com estratificação.

**Por que assim?**
- **Validação balanceada** garante comparação justa entre classes.
- **Teste separado** permite comparar métodos antes da avaliação final.
- **Treino maior** melhora a qualidade dos modelos.


In [1]:
from classifier.data import load_dataset, train_test_validation_split

# Carregar dataset
df, classes = load_dataset()
print(f"Total de tickets: {len(df):,}")
df.head()


Total de tickets: 47,837


,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous


In [2]:
# Classes obtidas do dataset
print(f"Classes ({len(classes)}):")
for c in classes:
    print(f"  - {c}")

Classes (8):
  - Access
  - Administrative rights
  - HR Support
  - Hardware
  - Internal Project
  - Miscellaneous
  - Purchase
  - Storage


In [3]:
from classifier.config import VALIDATION_SIZE

# Split: validação balanceada + treino/teste estratificados (80/20)
train_df, test_df, validation_df = train_test_validation_split(
    df,
    validation_size=VALIDATION_SIZE,
)

print(f"Treino:     {len(train_df):,} tickets")
print(f"Teste:      {len(test_df):,} tickets")
print(f"Validação:  {len(validation_df):,} tickets")
print("\nDistribuição na validação:")
print(validation_df["Topic_group"].value_counts().sort_index())

Treino:     38,109 tickets
Teste:      9,528 tickets
Validação:  200 tickets

Distribuição na validação:
Topic_group
Access                   25
Administrative rights    25
HR Support               25
Hardware                 25
Internal Project         25
Miscellaneous            25
Purchase                 25
Storage                  25
Name: count, dtype: int64


## 2. Métodos de Classificação e Comparação

Os experimentos de classificação estão em `notebooks/classificators.ipynb`, onde rodamos TF-IDF, embeddings e RAG no conjunto de teste.

Modelo selecionado (teste):
- **TF-IDF + LinearSVC** — accuracy **0.8637**, F1 macro **0.8641**, F1 weighted **0.8639**.

As seções seguintes assumem este modelo como escolhido.


## 3. Design dos Prompts

O prompt é a interface entre nosso sistema e a LLM. Um bom design de prompt é crucial para obter classificações precisas e justificativas úteis.

### Estrutura do Prompt

O prompt é dividido em duas partes:

1. **System prompt:** Define o papel da LLM (classificador), lista as classes válidas e especifica o formato de saída (JSON)

2. **User prompt:** Contém o ticket a classificar e os exemplos de contexto

### Parâmetros Configuráveis

| Parâmetro | Descrição | Valor Padrão |
|-----------|-----------|---------------|
| `K_SIMILAR` | Número de tickets similares do RAG | 5 |
| `reference_tickets` | Tickets representativos por classe | 1 por classe |

### Trade-offs

- **Mais exemplos similares:** Melhor contexto para classificação, mas aumenta tokens e custo
- **Tickets de referência:** Garante diversidade de classes, essencial para justificativas comparativas
- **Muitos exemplos:** Pode "poluir" o contexto e confundir a LLM

Vamos visualizar como os prompts são gerados:

In [ ]:
from classifier.prompts import build_system_prompt, build_user_prompt

In [ ]:
# Exemplo 1: System Prompt
system_prompt = build_system_prompt(classes)
print("=== SYSTEM PROMPT ===\n")
print(system_prompt)

In [ ]:
# Exemplo 2: User Prompt COM tickets de referência
user_prompt_with_refs = build_user_prompt(query, similar, representatives)
print("=== USER PROMPT (com tickets de referência) ===\n")
print(user_prompt_with_refs)

In [ ]:
# Exemplo 3: User Prompt SEM tickets de referência
# Útil quando queremos usar menos tokens ou quando os similares já são suficientes
user_prompt_no_refs = build_user_prompt(query, similar, reference_tickets=None)
print("=== USER PROMPT (sem tickets de referência) ===\n")
print(user_prompt_no_refs)

### Comparação de Uso de Tokens

A tabela abaixo mostra o impacto real de cada parâmetro no consumo de tokens. Isso ajuda a escolher a configuração ideal considerando o trade-off entre qualidade do contexto e custo/latência.

In [ ]:
import tiktoken

# Usar tokenizer cl100k_base (compatível com GPT-4, GPT-3.5-turbo, etc.)
enc = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    """Conta tokens usando o tokenizer cl100k_base."""
    return len(enc.encode(text))

# Comparar diferentes configurações de prompt
# K=5 sem refs é a baseline (configuração mínima recomendada)
configs = [
    ("K=5, sem refs", build_user_prompt(query, similar[:5], None)),
    ("K=5, com refs", build_user_prompt(query, similar[:5], representatives)),
    ("K=3, sem refs", build_user_prompt(query, similar[:3], None)),
    ("K=3, com refs", build_user_prompt(query, similar[:3], representatives)),
    ("K=1, sem refs", build_user_prompt(query, similar[:1], None)),
    ("K=1, com refs", build_user_prompt(query, similar[:1], representatives)),
]

# Tabela comparativa
system_tokens = count_tokens(system_prompt)
print(f"System prompt: {system_tokens} tokens (fixo)\n")
print(f"{'Configuração':<16} | {'User Prompt':>12} | {'Total':>8} | {'vs baseline':>12}")
print("-" * 58)
baseline = None
for name, prompt in configs:
    user_tokens = count_tokens(prompt)
    total = system_tokens + user_tokens
    if baseline is None:
        baseline = total
        diff = "(base)"
    else:
        diff = f"{(total - baseline) / baseline * 100:+.0f}%"
    print(f"{name:<16} | {user_tokens:>12} | {total:>8} | {diff:>12}")

## 4. Classificação com LLM

Com o prompt construído, enviamos para a LLM. O sistema suporta qualquer API compatível com OpenAI configurada via variáveis de ambiente.

**Requer:** variáveis de ambiente `LLM_BASE_URL` e `LLM_MODEL` (ver `.env.example`)

In [ ]:
from classifier.llm import TicketClassifier

In [ ]:
# Inicializar classificador
classifier = TicketClassifier()

In [ ]:
# Classificar o ticket de teste
details = classifier.classify(query, similar, classes, reference_tickets=representatives)

print(f"Classe real: {true_class}")
print(f"Classe predita: {details.result.classe}")
print(f"Justificativa: {details.result.justificativa}")
if details.reasoning:
    print(f"\nReasoning:")
    print(details.reasoning)
print(f"\nCorreto: {details.result.classe == true_class}")

#### Exemplo de Resposta com Reasoning

**Sem reasoning:**
```json
{
  "classe": "Access",
  "justificativa": "O ticket menciona problemas com cartão de acesso."
}
```

**Com reasoning:**
```json
{
  "classe": "Access",
  "justificativa": "O ticket descreve problemas com 'card key' e 'door access'...",
  "reasoning": "Analisei o ticket e identifiquei palavras-chave como 'card key', 'door' e 'access'..."
}
```

#### Configuração

```bash
# .env
LLM_MODEL=xiaomi/mimo-v2-flash:free
LLM_REASONING_EFFORT=medium  # low, medium, ou high
```

Via CLI:
```bash
uv run python main.py --reasoning medium
```

### Uso de Reasoning

Ativamos o **modo reasoning** do modelo (`LLM_REASONING_EFFORT=medium`) para melhorar a qualidade das classificações.

#### Benefícios

| Aspecto | Impacto do Reasoning |
|---------|---------------------|
| **Casos ambíguos** | O modelo "pensa" antes de decidir, analisando nuances |
| **Justificativas** | Explicações mais detalhadas do processo de decisão |
| **Precisão** | Reduz erros entre classes similares |
| **Transparência** | Campo `reasoning` mostra o raciocínio completo |

#### Trade-offs

Ao ativar reasoning, enfrentamos:
| Aspecto | Trade-off |
|---------|-----------|
| **Tokens** | Aumento no número de tokens e custo por chamada |
| **Latência** | Respostas mais lentas devido ao processamento adicional |

## 4.5 Escolha do Modelo LLM

Foi escolhido o modelo **MIMO v2 Flash** da Xiaomi, baseado nos seguintes critérios:

- Disponível gratuitamente via OpenRouter
- Suporte nativo a reasoning
- Performance competitiva

### Arquitetura da Solução

O diagrama abaixo ilustra o fluxo completo de classificação:

In [ ]:
import plotly.graph_objects as go

# Criar diagrama de arquitetura
fig = go.Figure()

# Definir as etapas do pipeline
steps = [
    ("Ticket", "#e1f5fe", "Texto de entrada"),
    ("Embedding", "#fff3e0", "all-MiniLM-L6-v2\n384 dimensões"),
    ("Retrieval", "#e8f5e9", "K similares +\nRepresentativos"),
    ("Prompt", "#fce4ec", "System + User\n+ Contexto RAG"),
    ("LLM", "#f3e5f5", "API OpenAI-compatible\n+ Retry JSON"),
    ("Output", "#e0f2f1", '{"classe": "...",\n"justificativa": "..."}'),
]

# Posições
x_positions = list(range(len(steps)))
y_pos = 0.5

# Adicionar caixas e textos
for i, (name, color, desc) in enumerate(steps):
    # Caixa
    fig.add_shape(
        type="rect",
        x0=i - 0.4, x1=i + 0.4,
        y0=0.2, y1=0.8,
        fillcolor=color,
        line=dict(color="#333", width=2),
    )
    # Nome da etapa
    fig.add_annotation(
        x=i, y=0.65,
        text=f"<b>{name}</b>",
        showarrow=False,
        font=dict(size=14),
    )
    # Descrição
    fig.add_annotation(
        x=i, y=0.38,
        text=desc,
        showarrow=False,
        font=dict(size=10),
        align="center",
    )
    # Seta para próxima etapa
    if i < len(steps) - 1:
        fig.add_annotation(
            x=i + 0.5, y=0.5,
            ax=i + 0.42, ay=0.5,
            xref="x", yref="y",
            axref="x", ayref="y",
            showarrow=True,
            arrowhead=2,
            arrowsize=1.5,
            arrowcolor="#333",
        )

fig.update_layout(
    title=dict(text="Arquitetura do Pipeline RAG", x=0.5, font=dict(size=16)),
    xaxis=dict(visible=False, range=[-0.6, len(steps) - 0.4]),
    yaxis=dict(visible=False, range=[0, 1]),
    height=250,
    margin=dict(l=20, r=20, t=50, b=20),
    plot_bgcolor="white",
)

fig.show()

## 5. Pipeline Completo com LangGraph

O LangGraph orquestra todo o fluxo de classificação em um grafo de estados com 4 nós:

| Nó | Função |
|----|--------|
| **embed** | Gera embedding do ticket (384 dimensões) |
| **retrieve** | Busca K tickets similares usando o embedding |
| **build_prompt** | Constrói system e user prompts com contexto RAG |
| **classify** | Chama a LLM e processa a resposta JSON |

Abaixo visualizamos a estrutura do grafo:

In [ ]:
from IPython.display import Image, display
from classifier.graph import create_graph

# Criar o grafo para visualização
pipeline = create_graph(retriever, classifier, classes, representatives)

# Visualizar a estrutura do grafo LangGraph
display(Image(pipeline.get_graph().draw_mermaid_png()))

In [ ]:
from classifier.graph import classify_ticket

In [ ]:
# Classificar usando o pipeline completo
# A função classify_ticket encapsula todo o fluxo: retrieve → classify
details_graph = classify_ticket(
    ticket=query,
    retriever=retriever,
    classifier=classifier,
    classes=classes,
    reference_tickets=representatives,
)

print(f"Classe real: {true_class}")
print(f"Classe predita: {details_graph.result.classe}")
print(f"Justificativa: {details_graph.result.justificativa}")
if details_graph.reasoning:
    print(f"\nReasoning:")
    print(details_graph.reasoning)
print(f"\nCorreto: {details_graph.result.classe == true_class}")

## 6. Avaliação do Classificador

Agora vamos avaliar o desempenho do classificador nos 200 tickets de teste.

### Métricas Utilizadas

- **Accuracy:** Proporção de classificações corretas
- **F1 Macro:** Média não-ponderada do F1 por classe (trata todas as classes igualmente, importante para datasets desbalanceados)
- **Cohen's Kappa:** Mede concordância além do acaso (valores próximos de 1 indicam excelente concordância)
- **MCC (Matthews Correlation Coefficient):** Métrica robusta para classificação multi-classe
- **Confusion Matrix:** Visualiza erros de classificação entre classes

### Tratamento de Erros

LLMs podem ocasionalmente retornar JSON malformado, ou a API pode apresentar erros. Para lidar com isso:

**Erros de JSON (parsing):**
1. **Primeira tentativa:** Envia o prompt normal para a LLM
2. **Se JSON inválido:** Continua a conversa adicionando a resposta do assistant e um prompt de correção:
   ```
   Sua resposta anterior não está no formato JSON válido.
   Por favor, responda APENAS com JSON válido no formato:
   {"classe": "<categoria>", "justificativa": "<explicação>"}
   ```
3. **Segunda tentativa:** LLM recebe o contexto completo da conversa e tenta corrigir
4. **Se falhar novamente:** O ticket é marcado como erro e reportado separadamente

**Erros de API:**
- Erros de autenticação (401), rate limit (429), ou outros erros da API são capturados
- O ticket é marcado como erro com o motivo específico
- A avaliação continua com os próximos tickets

Este approach garante que:
- Um erro pontual não interrompe toda a avaliação
- A LLM tem chance de se corrigir com contexto adicional
- Erros persistentes são documentados para análise posterior

In [ ]:
from classifier.metrics import evaluate, print_report, plot_confusion_matrix, plot_per_class_metrics
from classifier.runner import classify_batch

In [ ]:
# Classificar todos os tickets de teste usando classify_batch()
results, classification_errors, total_tokens = classify_batch(
    test_df=test_df,
    retriever=retriever,
    classifier=classifier,
    classes=classes,
    reference_tickets=representatives,
    show_progress=True,
)

# Extrair y_true e y_pred para métricas
y_true = [r["true_class"] for r in results]
y_pred = [r["predicted_class"] for r in results]

print(f"\nClassificação concluída!")
print(f"Classificados com sucesso: {len(results)}")
if classification_errors:
    print(f"Erros de classificação (JSON inválido): {len(classification_errors)}")

print(f"\n{'='*60}")
print("TOKEN USAGE SUMMARY")
print(f"{'='*60}")
print(f"Prompt tokens:     {total_tokens.prompt_tokens:,}")
print(f"Completion tokens: {total_tokens.completion_tokens:,}")
print(f"Total tokens:      {total_tokens.total_tokens:,}")

In [ ]:
# Calcular métricas
metrics = evaluate(y_true, y_pred, classes)
print_report(metrics, classes)

### Interpretação das Métricas Gerais

Os resultados da avaliação indicam um classificador com **performance sólida**:

| Métrica | Valor | Interpretação |
|---------|-------|---------------|
| **Accuracy** | 76.5% | ~3 em 4 tickets classificados corretamente |
| **F1 Macro** | 75.8% | Performance equilibrada entre classes (não favorece majoritárias) |
| **Cohen's Kappa** | 0.73 | Concordância **substancial** (escala: >0.6 bom, >0.8 excelente) |
| **MCC** | 0.74 | Confirma robustez - métrica equilibrada para problemas multiclasse |

**Destaques por classe:**
- **Maior recall:** Access (92%) e Internal Project (92%) - o modelo raramente perde tickets dessas classes
- **Maior precision:** Purchase (95%) - quando prediz Purchase, quase sempre acerta
- **Classe problemática:** Administrative rights (F1=49%) - recall de apenas 36%, indicando que muitos tickets dessa classe são classificados incorretamente

### Baseline: RAG-only com votacao ponderada

Avaliamos o desempenho da classificacao direta via retrieval, sem LLM,
usando a mesma configuracao de K similares.


### Comparação: RAG+LLM vs Baseline (Weighted Vote)

| Aspecto | RAG + LLM | Baseline |
|---------|-----------|----------|
| Accuracy | 76.5% | **77.5%** |
| F1 Macro | 75.8% | **77.6%** |
| Tokens consumidos | 366k | 0 |
| Tempo de execução | ~45 min | ~2 seg |
| Justificativas | **Sim** | Não |

**Insights:**

1. **Métricas numéricas:** A baseline supera ligeiramente o RAG+LLM. Isso indica que o retriever já captura bem a semântica dos tickets - a LLM não adiciona acurácia significativa.

2. **Valor agregado da LLM:** O diferencial está nas **justificativas em linguagem natural**. Para sistemas de suporte, explicar *por que* um ticket foi classificado é tão importante quanto a classificação em si.

3. **Trade-off custo/benefício:** A LLM consome tokens e tempo, mas oferece interpretabilidade. Em produção, uma abordagem híbrida pode ser ideal: usar weighted vote para triagem rápida e LLM apenas quando justificativa for necessária.

4. **Erros similares:** Ambos os métodos têm dificuldade com Administrative rights, sugerindo que o problema está na representação semântica dessa classe, não na capacidade da LLM.

In [ ]:
from classifier.config import K_SIMILAR
from tqdm import tqdm

weighted_preds = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Classificando (weighted vote)"):
    similar = retriever.retrieve(row["Document"], k=K_SIMILAR)
    vote = retriever.weighted_vote(similar)
    weighted_preds.append(vote["predicted_class"])

y_true_weighted = test_df["Topic_group"].tolist()
metrics_weighted = evaluate(y_true_weighted, weighted_preds, classes)
print_report(metrics_weighted, classes)


### Baseline: Graficos (weighted vote)

Visualizacao por classe e confusion matrix para a baseline RAG-only.


In [ ]:
# Graficos por classe - weighted vote
plot_per_class_metrics(
    y_true_weighted,
    weighted_preds,
    classes,
    title="Precision, Recall e F1-Score por Classe - Weighted Vote"
)


In [ ]:
# Confusion matrix - weighted vote
plot_confusion_matrix(
    metrics_weighted["confusion_matrix"],
    classes,
    title="Confusion Matrix - Weighted Vote"
)


### Métricas por Classe

O gráfico abaixo mostra precision, recall e F1-score para cada classe, facilitando a identificação visual de quais classes têm melhor/pior desempenho.

In [ ]:
# Gráfico de métricas por classe
plot_per_class_metrics(y_true, y_pred, classes, title="Precision, Recall e F1-Score por Classe")

In [ ]:
# Visualizar confusion matrix
plot_confusion_matrix(metrics["confusion_matrix"], classes, title="Confusion Matrix - Classificador RAG")

### Análise da Matriz de Confusão

A matriz de confusão normalizada revela padrões importantes:

**Classes bem separadas (diagonal forte):**
- **Internal Project (92%):** Tickets com códigos de projeto são distintivos
- **Purchase (84%):** Vocabulário específico (PO, purchase order) facilita identificação
- **Access (92%):** Padrões como "access card", "password reset" são claros

**Principais confusões:**

| Confusão | Ocorrências | Possível causa |
|----------|-------------|----------------|
| Admin rights → Hardware | 12 (48%) | Tickets de "upgrade" e "update" aparecem em ambas |
| Miscellaneous → HR Support | 4 | Tickets administrativos genéricos |
| Storage → Access | 3 | Ambas tratam de "acesso" a recursos |

**Observações:**
- **Administrative rights** é a classe mais problemática: apenas 36% de recall
- **Miscellaneous** funciona como classe "catch-all", capturando tickets que não se encaixam claramente em outras categorias
- A confusão Hardware/Admin rights sugere que upgrades de **software** (Admin rights) são confundidos com upgrades de **hardware**

### Confusion Matrix Normalizada

A versão normalizada mostra percentagens em vez de contagens absolutas, facilitando a identificação de taxas de erro por classe.

In [ ]:
# Confusion matrix normalizada (percentagens)
plot_confusion_matrix(
    metrics["confusion_matrix"], 
    classes, 
    title="Confusion Matrix - Normalizada por True Label",
    normalize=True
)

### Análise de Erros

Existem dois tipos de erros na avaliação:

1. **Erros de classificação:** A LLM retornou um JSON válido, mas a classe predita difere da classe real
2. **Erros de parsing:** A LLM não retornou JSON válido mesmo após retry (tickets não incluídos nas métricas)

Abaixo analisamos os erros de classificação para entender os padrões de confusão entre classes.

In [ ]:
# Filtrar erros
errors = [r for r in results if not r["correct"]]
print(f"Total de erros: {len(errors)} de {len(results)} ({100 * len(errors) / len(results):.1f}%)\n")

# Mostrar alguns exemplos de erros
print("=" * 80)
print("EXEMPLOS DE ERROS DE CLASSIFICAÇÃO")
print("=" * 80)

for i, error in enumerate(errors[:5], 1):
    print(f"\n--- Erro {i} ---")
    print(f"Classe real:     {error['true_class']}")
    print(f"Classe predita:  {error['predicted_class']}")
    print(f"Justificativa:   {error['justification']}")
    print(f"Ticket:          {error['ticket'][:200]}...")

### Padrões nos Erros de Classificação

Analisando os 47 erros (23.5% do total), identificamos padrões recorrentes:

**1. Administrative rights → Hardware (12 erros)**
Exemplos mostram tickets sobre "upgrade", "update", "performance issues" que semanticamente poderiam pertencer a ambas:
- *"oracle upgrade zones"* → classificado como Hardware
- *"windows upgrade failed"* → classificado como Hardware
- *"performance issues after upgrading endpoint tool"* → classificado como Hardware

**2. Miscellaneous → HR Support (4 erros)**
Tickets administrativos genéricos que mencionam aprovações ou configurações de usuário.

**3. Purchase → Internal Project (3 erros)**
Confusão entre códigos de projeto e ordens de compra (PO), já que ambos envolvem códigos e formulários.

**Insight principal:**
Os limites semânticos entre certas classes são **inerentemente ambíguos**. Um ticket sobre "upgrade do Windows" pode ser:
- **Administrative rights:** se for sobre permissões para fazer upgrade
- **Hardware:** se for sobre o processo técnico de upgrade

Essa ambiguidade está no próprio dataset, não apenas no classificador.

**Erros coerentes com julgamento humano:**
As justificativas geradas pelo modelo para os erros são **razoáveis e bem fundamentadas**. Um exemplo ilustrativo: o ticket *"performance issues... after upgrading endpoint tool takes up cpu makes work impossible android studio..."* (classe real: Administrative rights) foi classificado como Hardware com a justificativa *"O ticket relata problemas de desempenho devido ao consumo excessivo de CPU pela ferramenta de endpoint após uma atualização, impossibilitando o trabalho em ambientes como Android Studio, o que está relacionado a recursos de hardware"*. A menção a CPU, problemas de desempenho e impossibilidade de trabalhar são sintomas tipicamente associados a limitações de hardware — um classificador humano facilmente chegaria à mesma conclusão. Os erros não são aleatórios ou absurdos — são confusões compreensíveis em zonas de fronteira semântica, onde a própria definição das classes no dataset é ambígua.

In [ ]:
# Distribuição de erros por classe
from collections import Counter

print("Distribuição de erros por classe real:")
error_by_true = Counter(e["true_class"] for e in errors)
for cls, count in sorted(error_by_true.items(), key=lambda x: -x[1]):
    print(f"  {cls}: {count} erros")

print("\nConfusões mais comuns (real → predito):")
confusion_pairs = Counter((e["true_class"], e["predicted_class"]) for e in errors)
for (true, pred), count in confusion_pairs.most_common(10):
    print(f"  {true} → {pred}: {count}x")

### Relação entre Similaridade e Acertos

A análise dos scores de similaridade revela uma correlação clara:

| Predições | Similaridade Média | Similaridade Mediana |
|-----------|-------------------|---------------------|
| **Corretas** | 0.776 | 0.778 |
| **Incorretas** | 0.674 | 0.671 |

**Diferença: ~13% menor para predições incorretas**

**Interpretação:**
1. Quando o retriever encontra tickets **muito similares** (score alto), a classificação tende a ser correta
2. Tickets com **baixa similaridade** aos exemplos do treino são mais propensos a erros
3. Esses tickets "difíceis" provavelmente são:
   - Atípicos ou outliers dentro de sua classe
   - Semanticamente ambíguos (pertencem a zonas de fronteira entre classes)

### Distribuição de Erros por Classe

Visualização gráfica dos erros por classe real, facilitando a identificação de quais classes são mais difíceis de classificar corretamente.

In [ ]:
import plotly.express as px

# Gráfico de erros por classe
if errors:
    error_data = [{"Classe": cls, "Erros": count} for cls, count in error_by_true.items()]
    fig = px.bar(
        error_data, 
        x="Erros", 
        y="Classe", 
        orientation="h",
        title="Erros por Classe Real",
        color="Erros",
        color_continuous_scale="Reds"
    )
    fig.update_layout(height=350, showlegend=False)
    fig.show()
else:
    print("Nenhum erro de classificação para exibir!")

### Análise de Scores de Similaridade

Comparação dos scores de similaridade entre predições corretas e incorretas. Scores mais baixos em predições incorretas indicam que o retriever teve dificuldade em encontrar exemplos relevantes. Se os scores são similares, o problema está na classificação do LLM.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Calcular média dos scores de similaridade por predição
def mean_similarity(result):
    scores = [t['score'] for t in result['similar_tickets']]
    return np.mean(scores) if scores else 0

scores_correct = [mean_similarity(r) for r in results if r['correct']]
scores_incorrect = [mean_similarity(r) for r in results if not r['correct']]

# Box plot comparativo
fig = go.Figure()
fig.add_trace(go.Box(y=scores_correct, name='Corretos', marker_color='#2ca02c'))
fig.add_trace(go.Box(y=scores_incorrect, name='Incorretos', marker_color='#d62728'))

fig.update_layout(
    title='Distribuição de Scores de Similaridade (Média dos K Similares)',
    yaxis_title='Score de Similaridade',
    height=400
)
fig.show()

# Estatísticas
print(f"Corretos:   média={np.mean(scores_correct):.3f}, mediana={np.median(scores_correct):.3f}, n={len(scores_correct)}")
if scores_incorrect:
    print(f"Incorretos: média={np.mean(scores_incorrect):.3f}, mediana={np.median(scores_incorrect):.3f}, n={len(scores_incorrect)}")
else:
    print("Nenhuma predição incorreta para análise.")

### Análise de Similaridade por Classe em Predições Erradas

Para cada predição incorreta, comparamos a similaridade do ticket com:
- **Classe predita**: A classe que o modelo escolheu incorretamente
- **Classe real**: A classe correta do ticket

Isso ajuda a entender se o erro foi "justificado" pelo RAG (similaridade alta com a classe predita) ou se o LLM cometeu um erro apesar dos sinais do RAG.

In [ ]:
# Análise de similaridade por classe para predições erradas
if errors:
    # Para cada erro, calcular similaridade com classe predita e classe real
    similarity_data = []

    for error in errors:
        ticket_text = error['ticket']
        pred_class = error['predicted_class']
        true_class = error['true_class']

        # Similaridade com classe predita
        pred_sim = retriever.compute_class_similarity(ticket_text, pred_class, k=5)
        # Similaridade com classe real
        true_sim = retriever.compute_class_similarity(ticket_text, true_class, k=5)

        similarity_data.append({
            'ticket': ticket_text[:100] + '...',
            'pred_class': pred_class,
            'true_class': true_class,
            'similarity_pred': pred_sim['mean_score'],
            'similarity_true': true_sim['mean_score'],
            'difference': pred_sim['mean_score'] - true_sim['mean_score'],
        })

    sim_df = pd.DataFrame(similarity_data)

    # Visualização: Box plot comparando as duas similaridades
    fig = go.Figure()
    fig.add_trace(go.Box(
        y=sim_df['similarity_pred'],
        name='Classe Predita',
        marker_color='#d62728'
    ))
    fig.add_trace(go.Box(
        y=sim_df['similarity_true'],
        name='Classe Real',
        marker_color='#2ca02c'
    ))

    fig.update_layout(
        title='Similaridade por Classe em Predições Erradas',
        yaxis_title='Score de Similaridade (média top-K)',
        height=400
    )
    fig.show()

    # Estatísticas
    print(f"Similaridade com classe predita: média={sim_df['similarity_pred'].mean():.3f}, "
          f"mediana={sim_df['similarity_pred'].median():.3f}")
    print(f"Similaridade com classe real:   média={sim_df['similarity_true'].mean():.3f}, "
          f"mediana={sim_df['similarity_true'].median():.3f}")

    # Análise: Quantos erros têm maior similaridade com a predita?
    higher_pred_sim = (sim_df['similarity_pred'] > sim_df['similarity_true']).sum()
    print(f"\nErros com maior similaridade à classe predita: {higher_pred_sim}/{len(errors)} "
          f"({100*higher_pred_sim/len(errors):.1f}%)")
else:
    print("Nenhum erro de classificação para análise de similaridade.")

## 7. Conclusão automática com LLM

Nesta etapa, usamos um prompt de avaliação final para sintetizar os resultados.
Ela compõe o processo de solução como a última fase de interpretação dos resultados,
logo após o cálculo das métricas e a análise de erros.
Ele recebe:
- métricas globais e por classe
- matriz de confusão e principais confusões
- amostras de erros (até 20), com justificativa e tickets similares do RAG
- configuração da execução (k, modelo, random_state)

O objetivo é gerar um texto técnico com interpretação de desempenho, padrões de erro,
papel do RAG e recomendações de melhorias, sem inventar números.

A seguir, geramos a conclusão com os dados desta execução.

In [ ]:
from classifier.conclusion import (
    build_conclusion_payload,
    build_conclusion_system_prompt,
    build_conclusion_user_prompt,
)
from classifier.config import EMBEDDING_MODEL, K_SIMILAR, RANDOM_STATE
from classifier.llm import ConclusionError

payload = build_conclusion_payload(
    dataset="dataset.csv",
    classes=classes,
    test_size=len(test_df),
    k_similar=K_SIMILAR,
    use_references=bool(representatives),
    embedding_model=EMBEDDING_MODEL,
    llm_model=classifier.model,
    random_state=RANDOM_STATE,
    classifications=results,
    errors=classification_errors,
    metrics=metrics,
    token_usage=total_tokens,
    max_misclassified=20,
)

system_prompt = build_conclusion_system_prompt()
user_prompt = build_conclusion_user_prompt(payload)

try:
    conclusion_text, conclusion_usage = classifier.generate_conclusion(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
    )
    print(conclusion_text)
    print(f"\nTokens usados (conclusão): {conclusion_usage.total_tokens:,}")
except ConclusionError as exc:
    print(f"Falha ao gerar conclusão: {exc}")


# 8. Considerações Finais

Com base nas métricas globais e por classe apresentadas acima, o pipeline RAG+LLM demonstra que o
retrieval é um componente determinante para a qualidade da classificação. Quando o retriever
recupera exemplos consistentes, a predição tende a ser estável e a justificativa permanece coerente
com os sinais do ticket.

A análise da matriz de confusão e dos erros indica que as classes com maior sobreposição semântica
são as mais suscetíveis a confusão. Isso sugere que melhorias no retriever (modelo de embedding,
normalização e seleção de K) e no prompt podem reduzir ambiguidades e elevar o recall das classes
mais críticas.

A conclusão automática do LLM é uma etapa adicional para sintetizar os achados, mas não substitui
a análise manual: ela funciona como um resumo guiado pelos dados fornecidos e pode acelerar a
interpretação técnica, mantendo a revisão humana como referência principal.
